In [147]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import CRUD module
from CRUD import AnimalShelter

# Import base64 for image encoding
import base64

###########################
# Data Manipulation / Model
###########################

# Initialize CRUD class with necessary connection details
shelter = AnimalShelter()

# Fetch data from MongoDB and convert it to a DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ returns the '_id' column which can have an invalid object type 'ObjectID'
# This will cause issues with the data table, so we remove it from the DataFrame
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Ensure no duplicates in the DataFrame based on 'rec_num'
df.drop_duplicates(subset='rec_num', keep='first', inplace=True)

# Generate base64 encoded image in memory
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('utf-8')

# Create a mapping of rescue types to preferred breeds, sex, and training age
rescue_criteria = {
    "Water Rescue": {
        "breeds": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": "Intact Female",
        "age_min": 26,
        "age_max": 156
    },
    "Mountain or Wilderness Rescue": {
        "breeds": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 26,
        "age_max": 156
    },
    "Disaster or Individual Tracking Rescue": {
        "breeds": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 20,
        "age_max": 300
    }
}

#########################
# Dashboard Layout / View
#########################

# Create a Dash app instance
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    # Header with Company Logo and Link to SNHU
    html.Div([
        html.A([
            html.Center(html.Img(
                src='data:image/png;base64,{}'.format(encoded_image),  # Embed image in base64 format
                height='225px', width='225px'  
            ))
        ], href='https://www.snhu.edu', target='_blank',
           style={'padding': '0px'}), 
        html.Center(html.B(html.H1('Joey Grippi SNHU CS-340 Dashboard'))) # Title of the dashboard
    ], style={'padding': '0px'}),  

    html.Hr(),

    # Filtering Options
    html.Div([
        dcc.RadioItems(
            id='rescue-type-filter',  # ID for the radio items component
            options=[{'label': i, 'value': i} for i in rescue_criteria.keys()] + [{'label': 'Reset', 'value': 'Reset'}],
            value='Reset',  # Default selected value
            style={'padding': '10px'}
        )
    ], style={'textAlign': 'center'}),

    html.Hr(),

    # Data Table Configuration
    dash_table.DataTable(
        id='datatable-id',  # ID for the data table component
        columns=[{"name": i, "id": i} for i in df.columns],  # Define table columns based on DataFrame
        data=df.to_dict('records'),  # Convert DataFrame to dictionary format for display
        page_action='native',  # Native pagination
        page_size=10,  # Number of rows per page
        editable=True,  # Allow editing of table cells
        row_selectable='single',  # Allow single row selection
        filter_action='native',  # Native filtering
        sort_action='native',  # Native sorting
        selected_rows=[],  # No rows selected by default
        style_table={
            'overflowX': 'auto',  # Allow horizontal scrolling if needed
            'width': '100%',  # Ensure table uses full container width
        },
        style_cell={
            'whiteSpace': 'normal',  # Allow text to wrap within cells
            'height': 'auto',  # Automatically adjust row height based on content
            'overflow': 'hidden',  # Hide overflow content
            'textOverflow': 'ellipsis',  # Add ellipsis if text overflows
            'minWidth': 'auto',  # Allow columns to shrink to fit content
            'maxWidth': 'none',  # Remove any maximum width constraint
        }
    ),

    html.Br(),
    html.Hr(),
    
    # Map Container
    html.Div(
        id='map-id',  # ID for the map container
        style={
            'width': '100%',  # Make the map container full width
            'display': 'flex',
            'justifyContent': 'center',  # Center the map horizontally
            'padding': '0',
            'margin': '0 auto'
        },
    ),

    html.Hr(),

    # Layout for Charts under the Map
    html.Div([
        html.Div(
            dcc.Graph(id='pie-chart'),  # Pie chart component
            id='pie-chart-container',  # ID to control visibility of the pie chart
            style={
                'width': '50%',  # Set width to half of the parent container
                'display': 'none',  # Default to hidden
                'padding': '10px'
            }
        ),
        html.Div(
            dcc.Graph(id='bar-chart'),  # Bar chart component
            id='bar-chart-container',  # ID to control visibility of the bar chart
            style={
                'width': '50%',  # Set width to half of the parent container
                'display': 'none',  # Default to hidden
                'padding': '10px'
            }
        )
    ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'}),  # Align charts side by side

     html.Hr(id='charts-separator'),  # ID to control visibility of the separator line
    
    # Unique Identifier with Centering
    html.Div([
        html.H2('Joey Grippi SNHU CS-340 MongoDB Authentication')
    ], style={'textAlign': 'center', 'marginTop': '20px'})
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on selected rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-filter', 'value')]  # Trigger callback on rescue type filter change
)
def update_table(selected_rescue_type):
    if selected_rescue_type == 'Reset':
        # Show the full DataFrame if 'Reset' is selected
        filtered_df = df
    else:
        # Apply filter based on selected rescue type
        if selected_rescue_type in rescue_criteria:
            criteria = rescue_criteria[selected_rescue_type]
            breeds = criteria["breeds"]
            sex = criteria["sex"]
            age_min = criteria["age_min"]
            age_max = criteria["age_max"]

            # Filter DataFrame based on criteria
            filtered_df = df[
                df['breed'].isin(breeds) &
                (df['sex_upon_outcome'] == sex) &
                (df['age_upon_outcome_in_weeks'] >= age_min) &
                (df['age_upon_outcome_in_weeks'] <= age_max)
            ]
        else:
            filtered_df = df

    # Return filtered data as dictionary format for data table
    return filtered_df.to_dict('records')

# Callback to update the map based on selected row data from the data table
@app.callback(
    Output('map-id', 'children'),
    [Input('rescue-type-filter', 'value'),
     Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(selected_rescue_type, viewData, selected_rows):
    # Convert viewData into a DataFrame for processing
    dff = pd.DataFrame.from_dict(viewData)

    # Default map settings
    default_center = [30.75, -97.48]  # Default center of the map
    default_zoom = 10  # Default zoom level
    zoom_for_all_markers = 12  # Zoom level when displaying all markers
    padding_factor = 0.1  # Padding factor to ensure markers are not too close to the map edges

    # Check if a specific rescue type is selected
    if selected_rescue_type != 'Reset':
        if selected_rescue_type in rescue_criteria:
            # Retrieve criteria for the selected rescue type
            criteria = rescue_criteria[selected_rescue_type]
            breeds = criteria["breeds"]
            sex = criteria["sex"]
            age_min = criteria["age_min"]
            age_max = criteria["age_max"]

            # Filter DataFrame based on the selected rescue criteria
            filtered_df = df[
                df['breed'].isin(breeds) &
                (df['sex_upon_outcome'] == sex) &
                (df['age_upon_outcome_in_weeks'] >= age_min) &
                (df['age_upon_outcome_in_weeks'] <= age_max)
            ]

            if not filtered_df.empty:
                # Calculate bounds for the filtered data
                min_lat = filtered_df['location_lat'].min()
                max_lat = filtered_df['location_lat'].max()
                min_lon = filtered_df['location_long'].min()
                max_lon = filtered_df['location_long'].max()

                # Apply padding to the bounds to ensure markers are visible with some space around them
                lat_range = max_lat - min_lat
                lon_range = max_lon - min_lon
                lat_padding = lat_range * padding_factor
                lon_padding = lon_range * padding_factor

                # Adjust bounds with padding
                min_lat -= lat_padding
                max_lat += lat_padding
                min_lon -= lon_padding
                max_lon += lon_padding

                # Calculate the center point for the map based on adjusted bounds
                bounds = [[min_lat, min_lon], [max_lat, max_lon]]
                center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]

                # Create markers for each entry in the filtered DataFrame
                markers = [
                    dl.Marker(position=[row['location_lat'], row['location_long']],
                              children=[
                                  dl.Tooltip(row['breed']),
                                  dl.Popup([
                                      html.H1(row['name']),
                                      html.P(f"Breed: {row['breed']}")
                                  ])
                              ])
                    for _, row in filtered_df.iterrows()
                ]
                map_children = [
                    dl.TileLayer(id="base-layer-id")
                ] + markers

                # If a row is selected, override the center and zoom with the selected row data
                if selected_rows:
                    row_index = selected_rows[0]
                    if 0 <= row_index < len(filtered_df):
                        selected_row = filtered_df.iloc[row_index]
                        latitude = selected_row['location_lat']
                        longitude = selected_row['location_long']
                        return dl.Map(style={'width': '100%', 'height': '500px'},
                                      center=[latitude, longitude], zoom=zoom_for_all_markers,
                                      children=[
                                          dl.TileLayer(id="base-layer-id"),
                                          dl.Marker(position=[latitude, longitude],
                                                    children=[
                                                        dl.Tooltip(selected_row['breed']),
                                                        dl.Popup([
                                                            html.H1(selected_row['name']),
                                                            html.P(f"Breed: {selected_row['breed']}")
                                                        ])
                                                    ])
                                      ])
                # Return map showing all markers within the calculated bounds
                return dl.Map(style={'width': '100%', 'height': '500px'},
                              center=center, zoom=zoom_for_all_markers,
                              children=map_children,
                              bounds=bounds)
            else:
                # No data available after filtering
                center = default_center
                zoom = default_zoom
                markers = [dl.Marker(position=center, children=[dl.Tooltip("No data available")])]
        else:
            # Invalid rescue type selected
            center = default_center
            zoom = default_zoom
            markers = [dl.Marker(position=center, children=[dl.Tooltip("No data available")])]
    else:
        # Default view when 'Reset' is selected
        center = default_center
        zoom = default_zoom
        markers = [dl.Marker(position=center, children=[dl.Tooltip("Select a row to see details")])]

    # If a row is selected, use the row data to center and zoom the map
    if selected_rows:
        row_index = selected_rows[0]
        if 0 <= row_index < len(dff):
            selected_row = dff.iloc[row_index]
            latitude = selected_row['location_lat'] if 'location_lat' in dff.columns else default_center[0]
            longitude = selected_row['location_long'] if 'location_long' in dff.columns else default_center[1]
            animal_name = selected_row['name'] if 'name' in dff.columns else 'Unknown'
            animal_breed = selected_row['breed'] if 'breed' in dff.columns else 'Unknown'
            return dl.Map(style={'width': '100%', 'height': '500px'},
                          center=[latitude, longitude], zoom=12,
                          children=[
                              dl.TileLayer(id="base-layer-id"),
                              dl.Marker(position=[latitude, longitude],
                                        children=[
                                            dl.Tooltip(animal_breed),
                                            dl.Popup([
                                                html.H1(animal_name),
                                                html.P(f"Breed: {animal_breed}")
                                            ])
                                        ])
                          ])

    # Return map with default settings and markers
    return dl.Map(style={'width': '100%', 'height': '500px'},
                  center=center, zoom=zoom,
                  children=[
                      dl.TileLayer(id="base-layer-id")
                  ] + markers)

# Callback to update the pie chart and bar chart based on selected rescue type
@app.callback(
    [Output('pie-chart-container', 'style'),
     Output('bar-chart-container', 'style'),
     Output('charts-separator', 'style'),
     Output('pie-chart', 'figure'),
     Output('bar-chart', 'figure')],
    [Input('rescue-type-filter', 'value')]  # Trigger callback on rescue type filter change
)
def update_charts(selected_rescue_type):
    if selected_rescue_type == 'Reset':
        # Hide the charts and the separator line when "Reset" is selected
        pie_style = {'display': 'none'}
        bar_style = {'display': 'none'}
        separator_style = {'display': 'none'}
        pie_fig = {}
        bar_fig = {}
    else:
        if selected_rescue_type in rescue_criteria:
            criteria = rescue_criteria[selected_rescue_type]
            breeds = criteria["breeds"]
            sex = criteria["sex"]
            age_min = criteria["age_min"]
            age_max = criteria["age_max"]

            # Filter DataFrame based on rescue criteria
            filtered_df = df[
                df['breed'].isin(breeds) &
                (df['sex_upon_outcome'] == sex) &
                (df['age_upon_outcome_in_weeks'] >= age_min) &
                (df['age_upon_outcome_in_weeks'] <= age_max)
            ]
        else:
            filtered_df = df
            
        # Replace missing names with their animal_id
        filtered_df['name'] = filtered_df.apply(
            lambda row: row['name'] if isinstance(row['name'], str) and row['name'].strip() != '' else row['animal_id'],
            axis=1
        )
        
        # Show the charts and the separator line
        pie_style = {'width': '50%', 'display': 'inline-block', 'padding': '10px'}
        bar_style = {'width': '50%', 'display': 'inline-block', 'padding': '10px'}
        separator_style = {'display': 'block'}
        
        # Create the pie chart showing distribution of breeds
        pie_fig = px.pie(filtered_df, names='breed', title='Distribution of Breeds')

        # Round the age values to the nearest whole number for the bar chart
        filtered_df['age_upon_outcome_in_weeks'] = filtered_df['age_upon_outcome_in_weeks'].round()
        
        # Create the bar chart showing the age of each dog
        bar_fig = px.bar(filtered_df, x='name', y='age_upon_outcome_in_weeks',
                         title='Age of Each Dog',
                         labels={'age_upon_outcome_in_weeks': 'Age (weeks)', 'name': 'Dog Name'},
                         text='age_upon_outcome_in_weeks')

    # Return the styles and figures for the charts
    return pie_style, bar_style, separator_style, pie_fig, bar_fig

# Start the Dash server with debug mode
app.run_server(debug=True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Dash app running on http://127.0.0.1:26532/
